In [1]:
import json
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping


2022-12-04 18:56:29.320869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 18:56:29.495296: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 18:56:29.495342: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-04 18:56:30.056261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
input_file_dir = 'finalMergedData.json'

In [3]:
rawDf = pd.read_json(input_file_dir)
rawDf.info()
#print(rawDf.describe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50906 entries, 0 to 50905
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   topic       50906 non-null  object
 1   source      50906 non-null  object
 2   url         50906 non-null  object
 3   title       50906 non-null  object
 4   author      50906 non-null  object
 5   date        50906 non-null  object
 6   content     50906 non-null  object
 7   source_url  50906 non-null  object
 8   bias_text   50906 non-null  object
dtypes: object(9)
memory usage: 3.5+ MB


## get target data from source data

In [4]:
df = rawDf[["source","content","bias_text"]]
df['bias_text'] = df['bias_text'].str.capitalize()
df.head()

/tmp/ipykernel_902515/2101829330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bias_text'] = df['bias_text'].str.capitalize()


,source,content,bias_text
0,Townhall,"By a two-to-one margin, respondents who partic...",Right
1,Townhall,President Trump loves to tweet as a way to dir...,Right
2,New York Times - News,"For Secret Service Chief, Revelations Could Th...",Left
3,NPR Online News,Trump Ousts Embattled Campaign Manager\n\nEnla...,Center
4,BBC News,Media playback is unsupported on your device M...,Center


In [5]:
news_source = df['source'].unique()
print(news_source.shape)


(713,)


In [6]:
bias_label = df['bias_text'].unique()
print(bias_label)

['Right' 'Left' 'Center' 'Lean left' 'Lean right' 'Mixed'
 'Political news media bias rating: not rated']


In [7]:
df.groupby('bias_text').describe()

source         \
                                             count unique   
bias_text                                                   
Center                                       12745    292   
Lean left                                     4795     90   
Lean right                                    2078     53   
Left                                         15098    174   
Mixed                                           57     10   
Political news media bias rating: not rated     10      6   
Right                                        16123    179   

                                                                        \
                                                             top  freq   
bias_text                                                                
Center                                           NPR Online News  2012   
Lean left                                        Washington Post  1198   
Lean right                                   Washington Examiner   752   
Left                                              CNN (Web News)  2905   
Mixed                                              The Flip Side    18   
Political news media bias rating: not rated         Guest Writer     5   
Right                                           Washington Times  2886   

                                            content         \
                                              count unique   
bias_text                                                    
Center                                        12745  12691   
Lean left                                      4795   4610   
Lean right                                     2078   2043   
Left                                          15098  14938   
Mixed                                            57     57   
Political news media bias rating: not rated      10      9   
Right                                         16123  15600   

                                                                                                \
                                                                                           top   
bias_text                                                                                        
Center                                       Biden's ability to reshape U.S. judiciary hang...   
Lean left                                    Why did this happen?\n\nPlease make sure your ...   
Lean right                                                                                       
Left                                         The United Nations' weather agency says the st...   
Mixed                                        From the Left\n\nThe left opposes the decision...   
Political news media bias rating: not rated                                                      
Right                                                                                            

                                                  
                                            freq  
bias_text                                         
Center                                        19  
Lean left                                    118  
Lean right                                    34  
Left                                          79  
Mixed                                          1  
Political news media bias rating: not rated    2  
Right                                        337

In [8]:
df_filtered = df[df['bias_text'] != "Political news media bias rating: not rated"]  
df_filtered = df_filtered[df_filtered['bias_text'] != "Mixed"]
df_filtered = df_filtered.drop_duplicates(subset=['content'], keep='first')
df_filtered = df_filtered[(df_filtered['content'] != "")]
#df_filtered.describe()

In [9]:
df_filtered.groupby('bias_text').describe()

source                                   content         \
            count unique                  top  freq   count unique   
bias_text                                                            
Center      12687    286      NPR Online News  2012   12687  12687   
Lean left    4569     89      Washington Post  1181    4569   4569   
Lean right   2038     47  Washington Examiner   751    2038   2038   
Left        14937    173       CNN (Web News)  2905   14937  14937   
Right       15594    176     Washington Times  2884   15594  15594   

                                                                    
                                                          top freq  
bias_text                                                           
Center      Trump Ousts Embattled Campaign Manager\n\nEnla...    1  
Lean left   Oprah rejects her protégé Dr. Oz and backs Dem...    1  
Lean right  NEW You can now listen to Fox News articles!\n...    1  
Left        For Secret Service Chief, Revelations Could Th...    1  
Right       By a two-to-one margin, respondents who partic...    1

In [10]:
df_filtered = df_filtered[df_filtered['bias_text'] != "Lean left"]
df_filtered = df_filtered[df_filtered['bias_text'] != "Lean right"]
df_filtered.groupby('bias_text').describe()

source                                content         \
           count unique               top  freq   count unique   
bias_text                                                        
Center     12687    286   NPR Online News  2012   12687  12687   
Left       14937    173    CNN (Web News)  2905   14937  14937   
Right      15594    176  Washington Times  2884   15594  15594   

                                                                   
                                                         top freq  
bias_text                                                          
Center     Trump Ousts Embattled Campaign Manager\n\nEnla...    1  
Left       For Secret Service Chief, Revelations Could Th...    1  
Right      By a two-to-one margin, respondents who partic...    1

## begin cleaning text

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/dkang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text) # this removes all special characters # r means raw string
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text
df_filtered['content'] = df_filtered['content'].apply(clean_text)
# remove all digits
df_filtered['content'] = df_filtered['content'].str.replace('\d+', '')

df_filtered.head()

/tmp/ipykernel_902515/1015252412.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df_filtered['content'] = df_filtered['content'].str.replace('\d+', '')


,source,content,bias_text
0,Townhall,twotoone margin respondents participated recen...,Right
1,Townhall,president trump loves tweet way directly reach...,Right
2,New York Times - News,secret service chief revelations could threate...,Left
3,NPR Online News,trump ousts embattled campaign managerenlarge ...,Center
4,BBC News,media playback unsupported device media captio...,Center


In [13]:
df_filtered.groupby('bias_text').describe()

source                                content         \
           count unique               top  freq   count unique   
bias_text                                                        
Center     12687    286   NPR Online News  2012   12687  12687   
Left       14937    173    CNN (Web News)  2905   14937  14934   
Right      15594    176  Washington Times  2884   15594  15589   

                                                                   
                                                         top freq  
bias_text                                                          
Center     trump ousts embattled campaign managerenlarge ...    1  
Left        years ago cnn democrats released party platfo...    2  
Right      texas sen ted cruz announced monday running re...    2

In [14]:
def countNumWordsOfContent(text):
  return len(text.split())

# remove those articles with actual words less than 30
df_filtered = df_filtered[df_filtered['content'].apply(countNumWordsOfContent) >= 50]


#a = df_filtered['content'].apply(countNumWordsOfContent)
#a.describe()

In [15]:
df_filtered.groupby('bias_text').describe()

source                                content         \
           count unique               top  freq   count unique   
bias_text                                                        
Center     12472    279   NPR Online News  2012   12472  12472   
Left       14889    173    CNN (Web News)  2905   14889  14886   
Right      15576    175  Washington Times  2884   15576  15571   

                                                                   
                                                         top freq  
bias_text                                                          
Center     trump ousts embattled campaign managerenlarge ...    1  
Left        years agowashington cnn new national poll sug...    2  
Right      south carolina sen lindsey graham called impro...    2

In [16]:
df_filtered = df_filtered.drop_duplicates(subset=['content'], keep='first')
df_filtered = df_filtered[(df_filtered['content'] != "")]
df_filtered.groupby('bias_text').describe()

source                                content         \
           count unique               top  freq   count unique   
bias_text                                                        
Center     12472    279   NPR Online News  2012   12472  12472   
Left       14886    173    CNN (Web News)  2905   14886  14886   
Right      15571    175  Washington Times  2884   15571  15571   

                                                                   
                                                         top freq  
bias_text                                                          
Center     trump ousts embattled campaign managerenlarge ...    1  
Left       secret service chief revelations could threate...    1  
Right      twotoone margin respondents participated recen...    1

## build LSTM Model

In [17]:
def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    print(len(list_set))

#allStrs = df_filtered['content'].values.tolist()

#allStr = []
#for str in allStrs:
#  allStr += str.split()

#unique(allStr)

# experiment 1 -- random split, not remove media source

In [18]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 689434
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 8800
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_filtered['content'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 687488 unique tokens.


In [19]:
X = tokenizer.texts_to_sequences(df_filtered['content'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (42929, 8800)


In [20]:
# convert categorical labels to numbers
Y = pd.get_dummies(df_filtered['bias_text']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (42929, 3)


In [21]:
# split data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(38636, 8800) (38636, 3)
(4293, 8800) (4293, 3)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.1)])

2022-12-04 18:57:12.343039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 18:57:12.343106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-04 18:57:12.343142: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-04 18:57:12.347870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-12-04 18:57:12.347916: W te

Epoch 1/5
 51/544 [=>............................] - ETA: 2:18:58 - loss: 1.0723 - accuracy: 0.4216

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to csv:  
hist_csv_file = 'history_BLSTM_experiment1.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
f = open("result_BLSTM_experiment1.txt", "w")
f.write('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
f.close()

In [ ]:
# plt.title('Loss')
# plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='test')
# plt.legend()
# plt.show()
# plt.savefig('loss_BLSTM_experiment1.png')

In [ ]:
# plt.title('Accuracy')
# plt.plot(history.history['acc'], label='train')
# plt.plot(history.history['val_acc'], label='test')
# plt.legend()
# plt.show()
# plt.savefig('accurate_BLSTM_experiment1.png')

# experiment 2 : random split, remove media source in content

In [ ]:
# #news_source is ndarray
# def clean_media_source(text):
#     text = ' '.join(word for word in text.split() if word not in news_source) # remove media sources from text
#     return text
# df_filtered['content'] = df_filtered['content'].apply(clean_media_source)

In [ ]:
# def unique(list1):
#     # insert the list to the set
#     list_set = set(list1)
#     print(len(list_set))

# allStrs = df_filtered['content'].values.tolist()

# allStr = []
# for str in allStrs:
#  allStr += str.split()

# unique(allStr)

In [ ]:
# # The maximum number of words to be used. (most frequent)
# MAX_NB_WORDS = 689434
# # Max number of words in each complaint.
# MAX_SEQUENCE_LENGTH = 8800
# # This is fixed.
# EMBEDDING_DIM = 100
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
# tokenizer.fit_on_texts(df_filtered['content'].values)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))
# X = tokenizer.texts_to_sequences(df_filtered['content'].values)
# X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
# print('Shape of data tensor:', X.shape)
# # convert categorical labels to numbers
# Y = pd.get_dummies(df_filtered['bias_text']).values
# print('Shape of label tensor:', Y.shape)
# # split data
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
# print(X_train.shape,Y_train.shape)
# print(X_test.shape,Y_test.shape)

In [ ]:
# model = Sequential()
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(3, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# epochs = 5
# batch_size = 64

# history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.1)])

In [ ]:
# accr = model.evaluate(X_test,Y_test)
# print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
# f = open("result_LSTM_experiment2.txt", "w")
# f.write('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
# f.close()

In [ ]:
# plt.title('Loss')
# plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='test')
# plt.legend()
# plt.show()
# plt.savefig('loss_LSTM_experiment2.png')

In [ ]:
# plt.title('Accuracy')
# plt.plot(history.history['acc'], label='train')
# plt.plot(history.history['val_acc'], label='test')
# plt.legend()
# plt.show()
# plt.savefig('accurate_LSTM_experiment2.png')